In [19]:
import pandas as pd
import time
import random
import json
from datetime import datetime
from pytz import timezone

# scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import bs4
import requests
from requests.exceptions import RequestException



from CHN.general import Scrapping


import warnings
warnings.filterwarnings('ignore')

In [20]:
scrap = Scrapping()

In [1]:
url = 'https://republica.gt/'
buscar = 'buscar/'

In [3]:
nombre = 'bernardo arevalo'
name = '%20'.join(nombre.split())

In [4]:
response = requests.get(
    url + buscar + name, 
    proxies={'http': None, 'https': None}, 
    timeout=60,
    verify=False
)

In [5]:
response.status_code

200

In [7]:
soup = BeautifulSoup(response.content, "html.parser")

In [8]:
with open('../data/republica/base.html', 'w') as rep:
    rep.write(soup.prettify())

In [14]:
def extraer_info_republica(soup) -> dict:
    return {
        "title": soup.select_one(".nota__titulo-item a").get_text(strip=True),
        "url": soup.select_one(".nota__titulo-item a")["href"],
        "category": soup.select_one(".nota__volanta--text").get_text(strip=True),
        "time": soup.select_one(".nota__fecha").get_text(strip=True)
    }

In [12]:
articulos = soup.find_all('article', class_='nota nota--linea nota')
resultado = [extraer_info_republica(articulo) for articulo in articulos]

In [10]:
paginas = soup.find_all('a', class_='number')

numeros = [int(element.text.strip()) for element in paginas if element.text.strip().isdigit()]

max_page = max(numeros)

### iterando por las páginas

In [15]:
full = []
print('Inicio...')
for i in range(1, max_page + 1):
    # agregamos el número de página
    page = f'/{i}/'
    print(f'Solicitando info pag {i}')
    
    try:
        # solicitamos la información de la pag
        response = requests.get(
            url + buscar + name + page, 
            proxies={'http': None, 'https': None}, 
            timeout=60,
            verify=False)
        print(f'Estado: {response.status_code}')

        # si se recibio respuesta correcta, parseamos
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # extraemos la información de los articulos
            articulos = soup.find_all("article", class_='nota nota--linea nota')
            resultado = [extraer_info_republica(articulo) for articulo in articulos]

            # juntamos la data en formato json
            data = {"page": i, "articulos": resultado}
            full.append(data)
    except RequestException as e:
        print(f'Error al procesar pagina {i}: {e}')
        continue


info = json.dumps({'epInvestiga': full}, indent=4, ensure_ascii=False)
with open(f"../data/republica/resultados_1_{nombre}.json", 'w', encoding='utf-8') as f:
    f.write(info)

Inicio...
Solicitando info pag 1
Estado: 200
Solicitando info pag 2
Estado: 200
Solicitando info pag 3
Estado: 200
Solicitando info pag 4
Estado: 200
Solicitando info pag 5
Estado: 200
Solicitando info pag 6
Estado: 200
Solicitando info pag 7
Estado: 200
Solicitando info pag 8
Estado: 200
Solicitando info pag 9
Estado: 200
Solicitando info pag 10
Estado: 200
Solicitando info pag 11
Estado: 200
Solicitando info pag 12
Estado: 200
Solicitando info pag 13
Estado: 200
Solicitando info pag 14
Estado: 200
Solicitando info pag 15
Estado: 200
Solicitando info pag 16
Estado: 200
Solicitando info pag 17
Estado: 200
Solicitando info pag 18
Estado: 200
Solicitando info pag 19
Estado: 200
Solicitando info pag 20
Estado: 200
Solicitando info pag 21
Estado: 200
Solicitando info pag 22
Estado: 200
Solicitando info pag 23
Estado: 200
Solicitando info pag 24
Estado: 200
Solicitando info pag 25
Estado: 200
Solicitando info pag 26
Estado: 200
Solicitando info pag 27
Estado: 200
Solicitando info pag 28
Est

#### para un articulo extaer el texto en especifico y analizarlo o darselo a chatgpt

In [16]:
url_art = 'https://republica.gt/politica/diputados-de-la-une-afin-a-sandra-torres-piden-interpelar-a-cuatro-ministros-de-arevalo--202512814220'

In [22]:
response = scrap.genRequest(url_art)

In [24]:
soup = BeautifulSoup(response.content, 'html.parser')

with open('../data/republica/articulo.html', 'w') as rep:
    rep.write(soup.prettify())

In [25]:
def info_articulo(soup) -> dict:
    # Extraer el título
    titulo = soup.find('h1')  # Supongamos que el título está en un <h1>
    titulo_text = titulo.get_text(strip=True) if titulo else 'No disponible'

    # Extraer el autor
    autor = soup.find('a', class_='autor-link')
    autor_text = autor.get_text(strip=True) if autor else 'No disponible'

    # Extraer la fecha
    fecha = soup.find('div', class_='entry--mobile__fecha')
    fecha_text = fecha.get_text(strip=True) if fecha else 'No disponible'

    # Extraer el contenido del artículo
    contenido = soup.find('div', class_='articulo__cuerpo')
    contenido_text = contenido.get_text(strip=True) if contenido else 'No disponible'

    # Crear un diccionario con la información
    articulo_info = {
        'titulo': titulo_text,
        'autor': autor_text,
        'fecha': fecha_text,
        'contenido': contenido_text
    }
    return articulo_info

In [29]:
with open('../data/republica/articulo1_prueba.json', 'w', encoding='utf-8') as art:
    json.dump(info_articulo(soup), art, ensure_ascii=False, indent=4)